**Code for plots**

In [ ]:
# library(tidyverse)
# library(ggridges)
# library("viridis")

# df = read_csv("/../movies.csv")

# genre <- 'Comedy'
# year_info <- c(1900, 1980)
# options(repr.plot.width = 10, repr.plot.height = 10)

# count_per_genre <- function(genre = 'Comedy', year_info = c(1950,2010)) {
    
#     # Function creates a plot of Number of movies produced vs. genre 
#     # in between years
#     #
#     # --------
#     # @param : 
#     #        genre : provide a genre to highlight
#     #        year_info : provide a list/vector with 2 values 
#     #                    which give the start and end year to sort by
#     #
#     # --------
#     # @Return :
#     #        a plot of 'Genre' vs. 'Number of movies produced'
#     #
#     # --------
#     # @Example : 
#     #        count_per_genre('Comedy', list(1990,2010))
#     #        count_per_genre('Drama', c(1990,2010))
#     # --------
    
#     #Filtered between the dates
#     A <- df %>% filter(between(df$year, 
#                            as.integer(year_info[1]), 
#                            as.integer(year_info[2])))

#     #Count number of movies per genre
#     A <- A %>%
#         drop_na(Major_Genre) %>%
#         group_by(Major_Genre) %>%
#         summarise(count_per_genre = n())

#     #Used for highlighting specific genre
#     A <- A %>%
#         mutate(to_highlight = ifelse(Major_Genre == genre,"Yes", "No"))

#     #Return the ggplot
#     ggplot(A, aes(Major_Genre, count_per_genre, fill = to_highlight)) +
#         geom_bar(stat= 'identity', position = 'dodge') +
#         scale_fill_manual(values = c("Yes" = "orange", "No" = "Grey"), guide = FALSE) +
#         labs(x = "Number of movies produced", y = 'Genre', title = 'Popularity of Genres') +
#         coord_flip()
# }
# count_per_genre(genre, year_info)

# average_box_office <- function(genre = 'Comedy', year_info = list(1980,2010)) {
    
#     # Function creates a plot of Number of movies produced vs. genre 
#     # in between years
#     #
#     # --------
#     # @param : 
#     #        genre : provide a genre to highlight
#     #        year_info : provide a list/vector with 2 values 
#     #                    which give the start and end year to sort by
#     #
#     # --------
#     # @Return :
#     #        a plot of 'Genre' vs. 'Number of movies produced'
#     #
#     # --------
#     # @Example : 
#     #        average_box_office('Comedy', list(1990,2010))
#     #        average_box_office('Drama', c(1990,2010))
#     # --------
    
#     #Filtered between the dates
#     B <- df %>% filter(between(df$year, 
#                            as.integer(year_info[1]), 
#                            as.integer(year_info[2])))

#     #Evaluate international gross, gather for tidy format
#     B <- B %>%
#         drop_na(Major_Genre) %>%
#         mutate(International_Gross = Worldwide_Gross - US_Gross) %>%
#         select(Major_Genre, year, US_Gross, International_Gross) %>%
#         gather(key = 'Gross', value = 'amount', -Major_Genre, -year) %>%
#         group_by(year, Gross) %>%
#         summarise(amount = mean(amount))

#     #Return the ggplot
#     B_plot <- ggplot(B, aes(year, amount, fill = Gross)) +
#         geom_bar(stat= 'identity', position = 'dodge') +
#         labs(x = "Year", y = 'Dollars', title = 'Average box office') 
    
#     B_plot

# }

# options(repr.plot.width = 15)
# average_box_office()

# violinplot <- function(genre = 'Comedy', year_info = list(1950,2010)) {
    
#     # Function creates a heatmap of profit ratio vs. IMDB rating
#     # in between years
#     #
#     # --------
#     # @param : 
#     #        genre : provide a genre to highlight
#     #        year_info : provide a list/vector with 2 values 
#     #                    which give the start and end year to sort by
#     #
#     # --------
#     # @Return :
#     #        a plot of 'Genre' vs. 'Number of movies produced'
#     #
#     # --------
#     # @Example : 
#     #        heatmap('Comedy', list(1990,2010))
#     #        heatmap('Drama', c(1990,2010))
#     # --------
    
#     #Filtered between the dates
#     C <- df %>% filter(between(df$year, 
#                            as.integer(year_info[1]), 
#                            as.integer(year_info[2])))

#     #Evaluate international gross, gather for tidy format
#     C <- C %>%
#         drop_na(Major_Genre) %>%
#         subset(profit_ratio <= 20) 
        


#     #Return the ggplot
#     C_plot <- ggplot(C, aes(y = Major_Genre , 
#                             x = IMDB_Rating,
#                             color = profit_ratio,
#                             fill = ..x..
#                             #fill 
#                            )) +
#                 geom_density_ridges_gradient(alpha = 0.2) +
#                 scale_fill_viridis(option = 'plasma', name = 'Profit ratio') +
#                 labs(x = 'IMDB rating', 
#                      y = 'Genre', 
#                      title = 'Critical reception vs. Genre',
#                      legend = 'Profit ratio')
#     C_plot 
# }
# violinplot()





In [1]:
library(tidyverse)
library(ggridges)
library("viridis")

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.2.1     v purrr   0.3.3
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   1.0.0     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'ggridges'


The following object is masked from 'package:ggplot2':

    scale_discrete_manual


Loading required package: viridisLite



In [2]:
df = read_csv("../movies.csv")

Warning message:
"Missing column names filled in: 'X1' [1]"
Parsed with column specification:
cols(
  .default = col_double(),
  Title = col_character(),
  Release_Date = col_date(format = ""),
  MPAA_Rating = col_character(),
  Distributor = col_character(),
  Source = col_character(),
  Major_Genre = col_character(),
  Creative_Type = col_character(),
  Director = col_character(),
  month_name = col_character(),
  day_of_week = col_character()
)

See spec(...) for full column specifications.



In [3]:
df %>% head()

X1,Title,US_Gross,Worldwide_Gross,US_DVD_Sales,Production_Budget,Release_Date,MPAA_Rating,Running_Time_min,Distributor,...,IMDB_Votes,year,month,month_name,day,day_of_week,rt_normalized,profit,profit_ratio,overseas_gross
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<chr>,<dbl>,<chr>,...,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,The Land Girls,146083,146083,NA,8000000,1998-06-12,R,NA,Gramercy,...,1071,1998,6,June,12,Friday,NA,-7853917,-0.9817396,0
1,"First Love, Last Rites",10876,10876,NA,300000,1998-08-07,R,NA,Strand,...,207,1998,8,August,7,Friday,NA,-289124,-0.9637467,0
2,I Married a Strange Person,203134,203134,NA,250000,1998-08-28,NA,NA,Lionsgate,...,865,1998,8,August,28,Friday,NA,-46866,-0.1874640,0
3,Let's Talk About Sex,373615,373615,NA,300000,1998-09-11,NA,NA,Fine Line,...,NA,1998,9,September,11,Friday,1.3,73615,0.2453833,0
4,Slam,1009819,1087521,NA,1000000,1998-10-09,R,NA,Trimark,...,165,1998,10,October,9,Friday,6.2,87521,0.0875210,77702
5,Mississippi Mermaid,24551,2624551,NA,1600000,1999-01-15,NA,NA,MGM,...,NA,1999,1,January,15,Friday,NA,1024551,0.6403444,2600000


In [5]:
year_info = list(1991,1994)
genre_input = 'Drama'

**Code for summary stats**

In [16]:
biggest_success <- function(year_info = list(1900,2010), genre_input = 'Drama'){
#     This function returns the greatest box office from the movies dataset based on filtering inputs, 
#         formatted to help create an info table
#     -------
#     Inputs :
#         year_info : List with 2 values, takes in the year information from the callback above
#                     [1970,1985]
#         genre_input : (To be programmed) : takes in the genre information from the callback above
#                     'Drama', 'Any'
#     -------
#     Returns
#         A specifically formatted string
#     -------       
    
    k <- df %>%
        filter(year >= year_info[1] & year <= year_info[2] ) 

    k <- k %>%
        mutate(profit = Worldwide_Gross - Production_Budget) %>%
        arrange(profit)

    topsuccessgross <- round(k[[1,'Worldwide_Gross']]/1000000 , 2)
    topsuccessbudget <- round(k[[1,'Production_Budget']]/1000000, 2)
    topsuccessname <- k[[1,'Title']]
    
    cat(paste0(" ======== " , genre_input , " Movies from " , year_info[1], ' to ' , year_info[2]  ,
            " ======== \nTop Grossing Film: " , topsuccessname , 
            "\nBox Office returns: $" , round(topsuccessgross, 2) , "M"))
    }

biggest_success()


 ======== Drama Movies from 1900 to 2010 ======== 
Top Grossing Film: Town & Country
Box Office returns: $10.36M

In [17]:
biggest_flop <- function(year_info = list(1900,2010), genre_input = 'Drama') {
    #'This function returns the biggest box office flop from the movies dataset based on filtering inputs, 
    #'    formatted to help create an info table
    #'---------
    #'Inputs :
    #'    year_info : List with 2 values, takes in the year information from the callback above
    #'                [1970,1985]
    #'    genre_input : (To be programmed) : takes in the genre information from the callback above
    #'                'Drama', 'Any'
    #'---------
    #'Returns
    #'   A specifically formatted string


          
    #Condition to have data between those years
    k <- df %>%
        filter(year >= year_info[1] & year <= year_info[2] ) 

    k <- k %>%
        mutate(profit = Worldwide_Gross - Production_Budget) %>%
        arrange(profit)

    topflopgross <- round(k[[1,'Worldwide_Gross']]/1000000 , 2)
    topflopbudget <- round(k[[1,'Production_Budget']]/1000000, 2)
    topflopname <- k[[1,'Title']]

    cat(paste0 (" ============== Biggest flop ============== " , 
            "\nTitle: " , topflopname , 
            "\nBox Office: $" , topflopgross , 
            " M\nBudget: $" , topflopbudget , " M"))

}

biggest_flop()

 ============== Biggest flop ============== 
Title: Town & Country
Box Office: $10.36 M
Budget: $105 M

In [18]:
how_big <- function(year_info = list(1900,2010), genre_input = 'Drama'){
    #This function returns the total box office from the movies dataset based on filtering inputs, 
    #    formatted to help create an info table
    
    #Inputs :
    #    year_info : List with 2 values, takes in the year information from the callback above
    #                [1970,1985]
    #    genre_input : (To be programmed) : takes in the genre information from the callback above
    #                'Drama', 'Any'
    #Returns
    #    A specifically formatted string
    
    #Condition to have data between those years
        #Condition to have data between those years
    k <- df %>%
        filter(year >= year_info[1] & year <= year_info[2] ) 

    k <- k %>%
        mutate(profit = Worldwide_Gross - Production_Budget)
    
    total_boxoffice <- sum(k$profit)/1000000000

    
    cat(paste0("=========== Total =========== " , "\nWorldwide box office: $" , total_boxoffice , "B" ))
}

how_big()

=========== Total =========== 
Worldwide box office: $NAB

In [19]:
average_returns <- function(year_info = list(1900,2010), genre_input = 'Drama'){
#     This function returns average total box office from the movies dataset based on filtering inputs, 
#         formatted to help create an info table
#     ------------
#     Inputs :
#         year_info : List with 2 values, takes in the year information from the callback above
#                     [1970,1985]
#         genre_input : (To be programmed) : takes in the genre information from the callback above
#                     'Drama', 'Any'
#     ------------
#     Returns
#         A specifically formatted string
          
    k <- df %>%
        filter(year >= year_info[1] & year <= year_info[2] ) 

    k <- k %>%
        mutate(profit = Worldwide_Gross - Production_Budget)
    
    avg_returns <- mean(k$profit, na.rm = TRUE)/1000000000
    
    cat(paste0("Average return on investment: $" , avg_returns , "M" ))
}

average_returns()

Average return on investment: $0.0543079805202131M